In [30]:
import pandas as pd
import numpy as np
import datetime as dt
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import funcoes
import itertools
from sklearn.metrics import accuracy_score


sns.set_style('darkgrid')
%matplotlib inline

In [27]:
class_names = ['queimada','não queimada']

In [2]:
def get_melhores_colunas(df_rank):
    label_grupo_bom = df_rank[df_rank[1]==df_rank[1].max()].labels.values[0]
    melhores_colunas = df_rank[df_rank['labels']==label_grupo_bom][0]
    cols = melhores_colunas.values.tolist()
    return cols
def get_rank(df_conhecimento):
    # Criando classificador RF
    rf_clas = RandomForestClassifier(n_jobs=8, n_estimators=100, max_features=5, random_state=0, max_depth=3, min_samples_split=5)
    rf_clas.fit(df_conhecimento[colunas_stats], df_conhecimento.verifica)
    
    # Criando rank colunas
    df_rank = pd.DataFrame(sorted(list(zip(colunas_stats,rf_clas.feature_importances_)), key=lambda x: x[1], reverse=True))
    
    # Divide colunas em 2 grupos
    dados = df_rank[[1]].values
    kmeans = KMeans(n_clusters=2, init='k-means++', n_init=10, max_iter=300, n_jobs=8)
    kmeans = kmeans.fit(dados)
    labels = kmeans.predict(dados)
    df_rank['labels'] = labels
    
    return df_rank

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [3]:
df = pd.read_csv('conhecimento_filtrado.csv')

df1 = df[df.verifica==1].copy().reset_index(drop=True)
df3_full = df[df.verifica==3].copy().reset_index(drop=True)

ids_3 = np.random.choice(df3_full.id, len(df1), replace=False)
df3 = df[df.id.isin(ids_3)].copy().reset_index(drop=True)

In [4]:
df_conhecimento = pd.concat([df1, df3])
print(df_conhecimento.shape)
df_conhecimento.head(2)

(25786, 13)


,ndvi,nbrl,dif_ndvi,dif_dnbrl,medianb2,medianb3,medianb4,medianb5,medianb6,medianb7,id_cena,id,verifica
0,0.218736,-0.000218,0.120296,0.267360,0.107310,0.093289,0.097317,0.154613,0.204524,0.155465,LS82210702016235,1474044,1
1,0.158400,-0.016396,0.190224,0.261095,0.122466,0.111157,0.126185,0.175812,0.221049,0.170493,LS82210702016235,1474696,1


In [5]:
total_v1 = len(df1)
print(total_v1)
total_v3 = len(df3)
print(total_v3)
total = len(df_conhecimento)
print(total)

12893
12893
25786


In [6]:
# Gera amostra para teste
parte_30 = int((total/100)*30)
ids_30 = np.random.choice(df_conhecimento.id, parte_30, replace=False)
teste_original = df_conhecimento[df_conhecimento.id.isin(ids_30)].copy().reset_index(drop=True)
teste = teste_original.copy()
teste['verifica'] = 0
print(len(teste_original))

7735


In [7]:
modelo = df_conhecimento[~df_conhecimento.id.isin(ids_30)].copy()
print(len(modelo))

18051


In [8]:
# check total
print((len(teste)+len(modelo))== total)

True


In [9]:
colunas_stats = ['ndvi', 'nbrl', 'dif_ndvi', 'dif_dnbrl', 'medianb2', 'medianb3', 'medianb4', 'medianb5', 'medianb6', 'medianb7']

In [10]:
print('Criando classificador RF')
rf_clas = RandomForestClassifier(n_jobs=8, n_estimators=100, max_features=5, random_state=0, max_depth=3, min_samples_split=5)
rf_clas.fit(modelo[colunas_stats], modelo.verifica)

Criando classificador RF


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features=5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=8,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [11]:
df_rank = get_rank(modelo)
df_rank

,0,1,labels
0,medianb5,0.507727,0
1,nbrl,0.269579,0
2,medianb3,0.079431,1
3,medianb6,0.075347,1
4,ndvi,0.036770,1
5,medianb2,0.022745,1
6,dif_ndvi,0.004298,1
7,medianb7,0.003057,1
8,medianb4,0.000998,1
9,dif_dnbrl,0.000049,1


In [12]:
melhores_colunas = get_melhores_colunas(df_rank)
melhores_colunas

['medianb5', 'nbrl']

In [13]:
print('RF - Classifica dado usando as melhores colunas')
rf_clas = RandomForestClassifier(n_jobs=8, n_estimators=100, max_features=len(melhores_colunas), random_state=0, max_depth=3, min_samples_split=5)
rf_clas.fit(modelo[melhores_colunas], modelo.verifica)
teste['rf'] = rf_clas.predict(teste[melhores_colunas])
teste['rf_1'] = rf_clas.predict_proba(teste[melhores_colunas])[:, 0]

RF - Classifica dado usando as melhores colunas


In [14]:
print('Classificando com NN')
nn_clas = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.00010, max_iter=200)
nn_clas.fit(modelo[melhores_colunas], modelo.verifica)
teste['nn'] = nn_clas.predict(teste[melhores_colunas])
teste['nn_1'] = nn_clas.predict_proba(teste[melhores_colunas])[:, 0]


Classificando com NN


In [15]:
print('Cria avaliacao usando (RF+NN)/2 - Arredondado para 3 casas decimais')
teste['avaliacao'] = (teste["rf_1"] + teste["nn_1"]) / 2
teste['avaliacao'] = teste['avaliacao'].apply(lambda x: round(x, 4))
teste = teste.sort_values('avaliacao', ascending=False).reset_index(drop=True)

Cria avaliacao usando (RF+NN)/2 - Arredondado para 3 casas decimais


In [16]:
teste['classificacao'] = teste.avaliacao.apply(lambda x: 1 if x > 0.9 else 3)
teste['classificacao'].value_counts()

3    4806
1    2929
Name: classificacao, dtype: int64

In [114]:
y_pred = teste.sort_values('id').reset_index(drop=True)
y_true = teste_original.sort_values('id').reset_index(drop=True)

In [116]:
r = pd.merge(teste[['id', 'classificacao']], teste_original[['id', 'verifica']], on='id')
r['v'] = r.classificacao.astype(str) + r.verifica.astype(str)
print(len(r))
print(len(teste))
print(len(teste_original))

7735
7735
7735


In [117]:
teste.sort_values('id').id.head()

4285    21961
4101    34932
4414    35186
519     35489
520     44837
Name: id, dtype: int64

In [118]:
teste_original.sort_values('id').id.head()

4978    21961
4890    34932
4891    35186
413     35489
415     44837
Name: id, dtype: int64

In [119]:
teste.classificacao.value_counts()

3    4806
1    2929
Name: classificacao, dtype: int64

In [120]:
teste_original.verifica.value_counts()

1    3870
3    3865
Name: verifica, dtype: int64

In [121]:
r.v.value_counts()

33    3805
11    2869
31    1001
13      60
Name: v, dtype: int64

In [122]:
v1 = len(r[(r.verifica==1)])
print(v1)
v3 = len(r[(r.verifica==3)])
print(v3)

3870
3865


In [123]:
def f(x, v):
    return int((x*100)/v)

In [124]:
v11 = len(r[(r.verifica==1) & (r.classificacao == 1)])
v33 = len(r[(r.verifica==3) & (r.classificacao == 3)])
v13 = len(r[(r.verifica==1) & (r.classificacao == 3)])
v31 = len(r[(r.verifica==3) & (r.classificacao == 1)])
print(v11, v13)
print(v31,v33)
print(f(v11, v1), f(v13, v1))
print(f(v31, v3),f(v33,v3))

2869 1001
60 3805
74 25
1 98
